<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/RAG%20using%20Kendra%20%26%20Langchain%20AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3 langchain langchain-pinecone


In [ ]:
import boto3
import os


boto3_bedrock = boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='',aws_secret_access_key='')

kendra_client=boto3.client('kendra',region_name='us-east-1',aws_access_key_id='',aws_secret_access_key='')

In [ ]:
from langchain_community.retrievers import AmazonKendraRetriever

retriever = AmazonKendraRetriever(index_id="d0215a1d-87f2-41de-906f-edd13da9fb62",client =kendra_client)



response=retriever.get_relevant_documents("How many major terror attacks happened in any city in India since 2014?")

response

In [ ]:
from langchain import PromptTemplate

RAG_PROMPT_TEMPLATE = '''Here is some important context which can help inform the questions the Human asks.
Make sure to not make anything up to answer the question if it is not provided in the context.


{context}


Human: {human_input}

Assistant:
'''
PROMPT = PromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

import json
human_input=input("Enter the question : ")
search_results =retriever.get_relevant_documents(human_input)
context_string = '\n\n'.join([f'Document {ind+1}: ' + i.page_content for ind, i in enumerate(search_results)])
prompt_data = PROMPT.format(human_input=human_input, context=context_string)
inputText=prompt_data
body_part=json.dumps({'inputText': inputText,
 'textGenerationConfig': {'maxTokenCount': 8192,
  'stopSequences': [],
  'temperature': 0,
  'topP': 1}})
response = boto3_bedrock.invoke_model(
    body=body_part,
    contentType="application/json",
    accept="application/json",
    modelId='amazon.titan-text-express-v1'
)
output_text=json.loads(response['body'].read())['results'][0]['outputText']
output_text